In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Load the data into a DataFrame
excel_file_path = # TODO: Add file path here
df = pd.read_excel(excel_file_path)

# Rename the columns
df.rename(columns={'Datetime': 'Datetime',
                   'POWER_CONSUMPTION/Chiller Electricity Rate/W': 'Pow',
                   'CWL_PRI_CW_FLOW/kg/s': 'FlowRate',
                   'OA_TEMP/C': 'TempDryBulb',
                   'OA_TEMP_WB/C': 'TempWetBulb',
                   'CWL_PRI_SW_TEMPSPT/C': 'Setpoint',
                   'CWL_PRI_RW_TEMP/C': 'TempEvapIn',
                   'CWL_PRI_SW_TEMP/C': 'TempEvapOut'
                   }, inplace=True)

print(df.head)

# Convert 'Datetime' to datetime object
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Extract Month, Day, and Hour into separate columns
df['Month'] = df['Datetime'].dt.month
df['Day'] = df['Datetime'].dt.day
df['Hour'] = df['Datetime'].dt.hour

# Handle missing values or inconsistencies in the data if needed
df = df.dropna()

# Create the 'FOG' and 'MSG' features
df['FOG'] = df['Pow'].diff(periods=5)
df['MSG'] = df['Pow'].diff().rolling(window=5).apply(lambda x: np.mean(x ** 2))

# Specify the features to be used as input including Month, Day, Hour
features = ['Month', 'Day', 'Hour', 'FlowRate', 'TempEvapIn', 'TempEvapOut', 'Setpoint', 'TempDryBulb', 'TempWetBulb', 'FOG', 'MSG']
X = df[features]

# Initialize the Random Forest models with parameters from the paper
rf_model1 = RandomForestRegressor(n_estimators=100, random_state=42, max_features='sqrt')
rf_model2 = RandomForestRegressor(n_estimators=100, random_state=42, max_features='sqrt')

# Train the models
rf_model1.fit(X, df['Pow'].shift(-1).fillna(df['Pow']))
rf_model2.fit(X, df['Pow'].shift(-2).fillna(df['Pow']))

# Make predictions
df['Prediction_RF (Pow+1)'] = rf_model1.predict(X)
df['Prediction_RF (Pow+2)'] = rf_model2.predict(X)

# Save the DataFrame back to the Excel file
# df.to_excel(excel_file_path, index=False)


<bound method NDFrame.head of                   Datetime   FlowRate  TempEvapIn  TempEvapOut  Setpoint  \
0      2018-01-01 00:02:00  66.238838   13.707865     12.00000      14.0   
1      2018-01-01 00:04:00   0.000000   13.971430     13.97143      14.0   
2      2018-01-01 00:06:00  33.119419   14.209403     14.00000      14.0   
3      2018-01-01 00:08:00  33.119419   14.599290     14.00000      14.0   
4      2018-01-01 00:10:00  33.119419   14.916185     14.00000      14.0   
...                    ...        ...         ...          ...       ...   
262795 2018-12-31 23:52:00  66.238838   15.823018     14.00000      14.0   
262796 2018-12-31 23:54:00  66.238838   15.849969     14.00000      14.0   
262797 2018-12-31 23:56:00  66.238838   15.862128     14.00000      14.0   
262798 2018-12-31 23:58:00  66.238838   15.855746     14.00000      14.0   
262799 2019-01-01 00:00:00  66.238838   15.853492     14.00000      14.0   

        TempDryBulb  TempWetBulb            Pow  
0      

In [3]:
df

,Datetime,FlowRate,TempEvapIn,TempEvapOut,Setpoint,TempDryBulb,TempWetBulb,Pow,Month,Day,Hour,FOG,MSG,Prediction_RF (Pow+1),Prediction_RF (Pow+2)
0,2018-01-01 00:02:00,66.238838,13.707865,12.00000,14.0,24.000000,23.999990,146364.702671,1,1,0,NaN,NaN,67270.034332,90337.326249
1,2018-01-01 00:04:00,0.000000,13.971430,13.97143,14.0,24.000000,23.999989,0.000000,1,1,0,NaN,NaN,84185.371332,85643.356713
2,2018-01-01 00:06:00,33.119419,14.209403,14.00000,14.0,24.000000,23.999997,35737.977401,1,1,0,NaN,NaN,78525.536671,81953.271983
3,2018-01-01 00:08:00,33.119419,14.599290,14.00000,14.0,24.000000,23.999988,52316.455266,1,1,0,NaN,NaN,80411.049060,82490.843589
4,2018-01-01 00:10:00,33.119419,14.916185,14.00000,14.0,24.000000,23.999996,78351.846935,1,1,0,NaN,NaN,81245.802678,82771.566913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262795,2018-12-31 23:52:00,66.238838,15.823018,14.00000,14.0,25.973333,24.570283,156030.244466,12,31,23,-2472.381784,1.594172e+06,158397.340553,158575.611462
262796,2018-12-31 23:54:00,66.238838,15.849969,14.00000,14.0,25.980000,24.567794,158247.855106,12,31,23,-629.057393,2.549713e+06,160475.701486,159686.828416
262797,2018-12-31 23:56:00,66.238838,15.862128,14.00000,14.0,25.986667,24.565295,159248.384194,12,31,23,835.248529,2.706907e+06,160475.701486,159686.828416
262798,2018-12-31 23:58:00,66.238838,15.855746,14.00000,14.0,25.993333,24.562797,158723.234918,12,31,23,1.404224,2.743005e+06,160475.701486,159686.828416
